In [1]:
import requests
from datetime import datetime
import pandas as pd
import json
import os
import ipywidgets as w
import pyautogui as pag #controle da tela
import pyperclip as pc #copiar e colar
import time
import webbrowser

In [19]:
###Cofre Advance - busca os depositos###

In [20]:
def Cofre_Depositos(Dt_Caixa,Tp_Deposito):
    mensagem = ""
    Dt_inicio = str(Dt_Caixa) + 'T00:00:00'
    Dt_fim = str(Dt_Caixa) + 'T23:10:00'
    url = "https://api.cofreadvance.com.br/ObterDepositos"
    user_data=   {
        "keyId": "d110b6db9ac55581008b9b11b148c341",
        "TerminalNumber":"ADV2717",
        "dataini" : Dt_inicio,
        "datafim" : Dt_fim}
    response = requests.post(url=url,json=user_data)
    if response.status_code < 200 and response.status_code > 299:
        #erro
        mensagem = 'Status code: '+ response.status_code +' Reason: ' + response.reason + ' Texto: '+ response.text
        return []
       # os._exit(0)
    depositos = json.loads(response.text)
    if len(depositos)==0:
        return []
        #os._exit(0)
    else:
        pd_depositos = pd.DataFrame(depositos)
    #formata os campos
    pd_depositos['QtTwoHundredBills'] =pd_depositos['QtTwoHundredBills'].astype(int)
    pd_depositos['QtHundredBills'] =pd_depositos['QtHundredBills'].astype(int)
    pd_depositos['QtFiftyBills'] =pd_depositos['QtFiftyBills'].astype(int)
    pd_depositos['QtTwentyBills'] =pd_depositos['QtTwentyBills'].astype(int)
    pd_depositos['QtTenBills'] =pd_depositos['QtTenBills'].astype(int)
    pd_depositos['Amount'] =pd_depositos['Amount'].astype(int)
    ##Cofre Advance - busca a coleta do dia
    url = "https://api.cofreadvance.com.br/ObterColetas"
    user_data={
        "keyId": "d110b6db9ac55581008b9b11b148c341",
        "TerminalNumber":"ADV2717",
        "dataini" : Dt_inicio,
        "datafim" : Dt_fim}
    response = requests.post(url=url,json=user_data) 
    
    if response.status_code >= 200 and response.status_code <= 299:
        #sucesso
        Coleta = json.loads(response.text)
        if len(Coleta)==0:
            mensagem ='Não há Coleta'
            if Tp_Deposito=='Coleta':  return []
        else:
            pd_Coleta = pd.DataFrame(Coleta)
            Dt_Coleta = pd_Coleta.loc[0,'CollectDate']

    else:
        mensagem = 'Status code: '+ response.status_code +' Reason: ' + response.reason + ' Texto: ' + response.text
        return []
       # os._exit(0)
  
    ## Filtra os depositos para coleta ou fechament
    if len(Coleta)!=0:
        if Tp_Deposito =='Fechamento': 
            pd_filtra_cf = pd_depositos['DepositDate'] > Dt_Coleta
        else: 
            pd_filtra_cf = pd_depositos['DepositDate'] <= Dt_Coleta
        pd_deposito_cf = pd_depositos[pd_filtra_cf]
    else: 
        pd_deposito_cf= pd_depositos    
  
    # display(pd_deposito_cf)
    #display(pd_depositos)
    dep_hoje_carro_forte= pd_deposito_cf.groupby(['UserReg']).agg(
        {'QtTwoHundredBills':'sum', 
         'QtHundredBills':'sum', 
         'QtFiftyBills':'sum', 
         'QtTwentyBills':'sum', 
         'QtTenBills':'sum', 
         'Amount':'sum'})
    return dep_hoje_carro_forte

In [21]:
#Pega operadora no Gerlot
def pegar_operadora(x,y):
    pag.doubleClick(x, y)
    pag.hotkey('ctrl','c')
    return pc.paste()

In [22]:
print('Integração do Advance e Gerlot')

Integração do Advance e Gerlot


In [23]:
wDt_Caixa =w.DatePicker(description ='Data do Caixa',disabled=False)
wTp_Deposito =w.RadioButtons(options=['Coleta', 'Fechamento'], value=None,description='Tipo:',disabled=False)
bt_OK = w.Button(description='Importar')
bt_Output = w.Output()

In [24]:
display(wDt_Caixa,wTp_Deposito)

DatePicker(value=None, description='Data do Caixa')

RadioButtons(description='Tipo:', options=('Coleta', 'Fechamento'), value=None)

In [7]:
def on_button_clicked(b):
    with bt_Output:
        if (wDt_Caixa.value =='None') or (wTp_Deposito.value =='None'):
            print('Preencha as informações acima!')
        else:
            lancamento = Cofre_Depositos(wDt_Caixa.value,wTp_Deposito.value)
            if len(lancamento)==0:
                print('Não a deposito ou algum erro ocorreu')
             #   os._exit(0)
            else:
                display(lancamento)
                print('Aguarde...')
                #abre o Gerlot
                url = 'https://www.gerlot.com.br/login?codigo_loterico=21001250-1'
                webbrowser.register('chrome',
                    None,
                    webbrowser.BackgroundBrowser("C://Program Files//Google//Chrome//Application//chrome.exe"))
                webbrowser.get('chrome').open(url)
                ##logar
                time.sleep(5)
                pag.click(558,265)
                pc.copy('adriana')  #usuario
                pag.hotkey('ctrl','v')
                pag.press('tab')
                pc.copy('123456')  #senha
                pag.hotkey('ctrl','v')
                pag.press('tab')
                pag.press('enter')
                #Entrar nos Caixas Abertos
                time.sleep(5)
                pag.doubleClick(548,230)
                pag.hotkey('ctrl','c')
                caixa_abertos = pc.paste()
                caixa_abertos = int(caixa_abertos)
                pag.click(x=574, y=250)
                i=1
                while i <= caixa_abertos:
                    time.sleep(5)
                    if i==1:           ##Point(x=306, y=271) caixa  1
                        caixa1 =pegar_operadora(306,271)
                        pd_filtro = lancamento.filter(like=caixa1, axis=0)
                        if (pd_filtro['QtTwoHundredBills'].count()!=0):
                            pag.click(x=1288, y=276)
                            pag.click(x=1219, y=317)
                            time.sleep(5)
                            EfetivaDeposito(pd_filtro,caixa1)  
                        time.sleep(10)
                        
                    elif i==2:         ##Point(x=308, y=319) caixa 2
                        caixa2 =pegar_operadora(309,386)
                        pd_filtro = lancamento.filter(like=caixa2, axis=0)
                        
                        if (pd_filtro['QtHundredBills'].count()!=0):
                            pag.click(x=1287, y=393)
                            pag.click(x=1233, y=430)
                            time.sleep(5)
                            EfetivaDeposito(pd_filtro,caixa2)     
                        time.sleep(10)
                    elif i==3:         ##Point(x=312, y=365) caixa 3
                        caixa3 =pegar_operadora(312,432)
                        pd_filtro = lancamento.filter(like=caixa3, axis=0)
                        if (pd_filtro['QtHundredBills'].count()!=0): 
                            pag.click(x=1279, y=444)
                            pag.click(x=1243, y=479)
                            time.sleep(5)
                            EfetivaDeposito(pd_filtro,caixa3)     
                        time.sleep(10)
                    elif i==4:         ##Point(x=311, y=417) caixa 4
                        caixa4 =pegar_operadora(311,484)
                        pd_filtro = lancamento.filter(like=caixa4, axis=0)
                        if (pd_filtro['QtHundredBills'].count()!=0): 
                            pag.click(x=1278, y=493)
                            pag.click(x=1272, y=531)
                            time.sleep(5)                   
                            EfetivaDeposito(pd_filtro,caixa4)     
                        time.sleep(10)

                    elif i==5:         ##Point(x=308, y=470) caixa 5
                        caixa5 =pegar_operadora(310,533)
                        pd_filtro = lancamento.filter(like=caixa5, axis=0)
                        if (pd_filtro['QtHundredBills'].count()!=0): 
                            pag.click(x=1278, y=546)
                            pag.click(x=1245, y=580)
                            time.sleep(5)
                            EfetivaDeposito(pd_filtro,caixa5)     
                        time.sleep(10)
                    
                    i += 1
            pag.hotkey('ctrl', 'w')
            print('Finalizado')
bt_OK.on_click(on_button_clicked)

In [26]:
display(bt_OK, bt_Output)

Button(description='Importar', style=ButtonStyle())

Output()

In [3]:
#preenche o deposito no Gerlot
def EfetivaDeposito(lanc, Caixa):
    pag.click(x=589, y=230)   #lacre
    time.sleep(5)
    if wTp_Deposito.value =='Fechamento': 
        pc.copy('17')
    else:
        pc.copy('13')  
    pag.hotkey('ctrl','v')
    pd_notas = pd.DataFrame(lanc)
    time.sleep(3)
    pag.click(x=547, y=321)   #200
    pc.copy(int(pd_notas['QtTwoHundredBills']))
    time.sleep(3)
    pag.hotkey('ctrl','v')
    pag.click(x=632, y=347)   #100 
    
    pc.copy(int(pd_notas['QtHundredBills']))
    time.sleep(3)
    pag.hotkey('ctrl','v')
    pag.click(x=677, y=382)   #50
    pc.copy(int(pd_notas['QtFiftyBills']))
    time.sleep(3)
    pag.hotkey('ctrl','v')
    pag.click(x=640, y=409)   #20
    pc.copy(int(pd_notas['QtTwentyBills']))
    time.sleep(3)
    pag.hotkey('ctrl','v')
    pag.click(x=672, y=444)   #10
    pc.copy(int(pd_notas['QtTenBills']))
    pag.hotkey('ctrl','v')
    time.sleep(3)
    pag.click(x=719, y=599)
